In [5]:
import sys, random
desaster_path = "/Users/geomando/Dropbox/github/SeaGrantSimulation"
sys.path.append(desaster_path)
import simpy
from simpy.util import start_delayed
import pandas as pd
import numpy as np
from desaster import entities, capitals, request, io, movement, search, rebuild

In [6]:
simulation = simpy.Environment()

write_story = True

In [7]:
def master_process(simulation, human_capital, financial_capital, entity, write_story):
        
    yield simulation.process(request.inspection(simulation, human_capital, entity.residence, entity, write_story))
    
    if entity.residence.damage_state != 'None':
        
        money_patience = 3650

        yield simulation.process(search.rebuild_money(simulation, human_capital, 
                                                        financial_capital, entity, 
                                                        money_patience, write_story))

        if entity.residence.damage_state == 'Complete':
            home_patience = 2000

            search_outcome = yield simulation.process(search.permanent_housing(simulation, entity, home_patience, housing_stock, human_capital, write_story))

            if search_outcome == 'Gave up':
                return

        if entity.residence.damage_state != 'None':
            yield simulation.process(request.engineering_assessment(simulation, human_capital, entity, write_story))

            yield simulation.process(request.permit(simulation, human_capital, entity, write_story))

            yield simulation.process(rebuild.home(simulation, human_capital, financial_capital, entity, write_story))
               
   

In [8]:
human_cap_data = pd.Series(
        {
        "Inspectors": 100,
        "Insurance Adjusters": 40,
        "FEMA Processors": 40,
        "Permit Processors": 100,
        "Contractors": 40,
        "Loan Processors": 40,
        "Engineers": 1
                 }
        )

human_capital = capitals.HumanCapital(simulation, human_cap_data) 

In [9]:
financial_cap_data = {"FEMA Aid": 35000000,
                     "Building Materials": 2000000}

financial_capital = capitals.FinancialCapital(simulation, financial_cap_data) 

In [10]:
file_path = "../inputs/household_inputs_test.csv"
households_df = pd.read_csv(file_path)

households_df.head()

,Name,Income,Savings,Insurance,Address,Occupancy,Cost,Bedrooms,Bathrooms,Area,Year Built,Value,Damage State
0,Alfred,30000,10000,0,62 That St,Mobile Home,100,1,1,700,1920,100000,Slight
1,Bruce,100000,1000000,10000000,720 This Rd,Single Family Dwelling,100000,6,5,5000,1920,10000000,Slight
2,Selena,10000,100,0,1001 Other Ave,Multi Family Dwelling,10,0,1,250,1960,10000,Moderate
3,Fish,50000,1000,550000,26000 Out There Lane,Single Family Dwelling,2000,4,2,2000,2010,800000,Moderate


In [11]:
file_path = "../inputs/housing_stock_test.csv"
housing_stock_df = pd.read_csv(file_path)

housing_stock_df

,Address,Occupancy,Cost,Bedrooms,Bathrooms,Area,Year Built,Value,Damage State
0,100 New Ave,Mobile Home,100,1,1,700,1920,99999,None
1,101 New Ave,Single Family Dwelling,100000,6,5,5000,1920,9999,None
2,102 New Ave,Multi Family Dwelling,10,0,1,250,1960,9999,None
3,103 New Ave,Single Family Dwelling,2000,4,2,2000,2010,800000,None
4,104 New Ave,Mobile Home,100,1,1,700,1920,100000,None
5,105 New Ave,Single Family Dwelling,100000,6,5,5000,1920,10000000,None
6,106 New Ave,Multi Family Dwelling,10,0,1,250,1960,9999,None
7,107 New Ave,Single Family Dwelling,2000,4,2,2000,2010,800000,None
8,108 New Ave,Mobile Home,100,1,1,700,1920,100000,Complete
9,109 New Ave,Single Family Dwelling,100000,6,5,5000,1920,10000000,Moderate


In [12]:
households = entities.importHouseholds(simulation, households_df, write_story)

housing_stock = capitals.importHousingStock(simulation, housing_stock_df)


In [13]:
# Do inspections on all of the vacant homes in the housing stock
for home in housing_stock.items:
    simulation.process(request.inspection(simulation, human_capital, home))

In [14]:
# Schedule an event that randomly fixes moderately or completely damaged homes in the vacant housing stock
# with probability = fix_probability

fix_probability = 1.0
fix_schedule = 100

start_delayed(simulation, rebuild.stock(simulation, housing_stock, fix_probability), fix_schedule)

<Process(starter) object at 0x113f1f978>

In [15]:
# Create a master process for each household to be modeled in the simulation
for i in range(len(households)):
    simulation.process(master_process(simulation, human_capital, financial_capital, households[i], write_story))

In [16]:
simulation.run()

8 homes in the vacant building stock were fixed on day 100.


In [17]:
households[0].story

['Alfred lives in a 1 bedroom Mobile Home at 62 That St worth $100,000. ',
 "Alfred's house was inspected 10 days after the event and suffered $2,000 of damage.",
 'Alfred already had enough money to rebuild (1:,.0f) and did not seek assistance. ',
 'Alfred received an engineering assessment 35 days after the event. ',
 'Alfred received permit approval 70 days after the event. ',
 "Alfred's home was repaired 72 days after the event, taking 2 days to repair. "]

In [18]:
households[1].story

['Bruce lives in a 6 bedroom Single Family Dwelling at 720 This Rd worth $10,000,000. ',
 "Bruce's house was inspected 10 days after the event and suffered $200,000 of damage.",
 'Bruce submitted an insurance claim 10 days after the event. ',
 'Bruce received a $200,000 insurance payout 25 days after the event. ',
 'It took Bruce 15 days to receive enough financial assistance and now has $1,200,000 to rebuild. ',
 'Bruce received an engineering assessment 60 days after the event. ',
 'Bruce received permit approval 95 days after the event. ',
 "Bruce's home was repaired 97 days after the event, taking 2 days to repair. "]

In [19]:
households[2].story

['Selena lives in a 0 bedroom Multi Family Dwelling at 1001 Other Ave worth $10,000. ',
 "Selena's house was inspected 10 days after the event and suffered $1,000 of damage.",
 'Selena submitted a request to FEMA 10 days after the event. ',
 'Selena received $1,000 from FEMA 30 days after the event. ',
 'It took Selena 20 days to receive enough financial assistance and now has $1,100 to rebuild. ',
 'Selena received an engineering assessment 110 days after the event. ',
 'Selena received permit approval 145 days after the event. ',
 "Selena's home was repaired 175 days after the event, taking 30 days to repair. "]

In [20]:
households[3].story

['Fish lives in a 4 bedroom Single Family Dwelling at 26000 Out There Lane worth $800,000. ',
 "Fish's house was inspected 10 days after the event and suffered $80,000 of damage.",
 'Fish submitted an insurance claim 10 days after the event. ',
 'Fish received a $80,000 insurance payout 25 days after the event. ',
 'It took Fish 15 days to receive enough financial assistance and now has $81,000 to rebuild. ',
 'Fish received an engineering assessment 85 days after the event. ',
 'Fish received permit approval 120 days after the event. ',
 "Fish's home was repaired 150 days after the event, taking 30 days to repair. "]

In [21]:
for item in housing_stock.items:
    print(item.address, item.value, item.occupancy, item.damage_state, item.damage_value)

100 New Ave 99999 Mobile Home None 0.0
101 New Ave 9999 Single Family Dwelling None 0.0
102 New Ave 9999 Multi Family Dwelling None 0.0
103 New Ave 800000 Single Family Dwelling None 0.0
104 New Ave 100000 Mobile Home None 0.0
105 New Ave 10000000 Single Family Dwelling None 0.0
106 New Ave 9999 Multi Family Dwelling None 0.0
107 New Ave 800000 Single Family Dwelling None 0.0
108 New Ave 100000 Mobile Home None 0.0
109 New Ave 10000000 Single Family Dwelling None 0.0
110 New Ave 10000 Multi Family Dwelling None 0.0
111 New Ave 800000 Single Family Dwelling None 0.0
112 New Ave 100000 Mobile Home None 0.0
113 New Ave 10000000 Single Family Dwelling None 0.0
114 New Ave 10000 Multi Family Dwelling None 0.0
115 New Ave 800000 Single Family Dwelling None 0.0
